In [2]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 

## Settings

In [3]:
data_dir = "./data/cat-dog"
batch_size = 4

## Utils

In [4]:
import random

def flip_lr(x): 
# horizontal flip transformation
    if random.random()>0.5:
        x = np.fliplr(x).copy()
    return x

## Dataset

In [5]:
from torch.utils.data import Dataset
from skimage import io, img_as_float32
import os

class CatDogDataset(Dataset):
    def __init__(self, data_dir, is_train=False, transform=None):
        self.data_dir = data_dir
        self.is_train = is_train
        if is_train:
            self.data_dir = os.path.join(data_dir, "train")
        else:
            self.data_dir = os.path.join(data_dir, "test")
        self.img_ids = os.listdir(self.data_dir)
        self.transform = transform
            
    def __len__(self):
        return len(self.img_ids)
    
    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img_p = os.path.join(self.data_dir, img_id)
        img = img_as_float32(io.imread(img_p)).transpose((2,0,1))
        if self.transform is not None:
            img = self.transform(img)
        
        out = {}
        out['name'] = img_id
        if self.is_train:
            out['img'] = img
            out['label'] = 0 if img_id.split("_")[1]=='cat' else 1
        else:
            out['img'] = img
            out['label'] = 0 if img_id.split("_")[1]=='cat' else 1
        
        return out

In [6]:
from torch.utils.data import DataLoader
dataset = CatDogDataset(data_dir, is_train=True, transform=flip_lr)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)
for x_i, x in enumerate(dataloader):
    print(x['name'])
    print(x['label'])
    print(x['img'].shape)
    break

['flickr_dog_000013.jpg', 'flickr_cat_000054.jpg', 'flickr_dog_000044.jpg', 'flickr_dog_000002.jpg']
tensor([1, 0, 1, 1])
torch.Size([4, 3, 512, 512])


## Model

In [12]:
class ClassifyCatDog(nn.Module):
    def __init__(self):
        super(ClassifyCatDog, self).__init__()
        # Encoder
        self.encoder = nn.ModuleList()
        self.encoder.append(nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, stride=1, padding=1))
        self.encoder.append(nn.ReLU())
        self.encoder.append(nn.MaxPool2d(8, 8))
        self.encoder.append(nn.Conv2d(4, 4, 3, stride=1, padding=1))
        self.encoder.append(nn.ReLU())
        self.encoder.append(nn.MaxPool2d(8, 8))
        # Decoder 1
        self.decoder_1 = nn.ModuleList()
        self.decoder_1.append(nn.Conv2d(4, 3, 3, stride=1, padding=1))
        self.decoder_1.append(nn.ReLU())
        self.decoder_1.append(nn.MaxPool2d(4, 4))
        # Decoder 2
        self.decoder_2 = nn.ModuleList()
        self.decoder_2.append(nn.Conv2d(4, 3, 3, stride=1, padding=1))
        self.decoder_2.append(nn.ReLU())
        self.decoder_2.append(nn.MaxPool2d(4, 4))
        # Classifier
        self.classifier = nn.ModuleList()
        self.classifier.append(nn.Linear(3*2*2, 4))
        self.classifier.append(nn.ReLU())
        self.classifier.append(nn.Linear(4, 2))
    def forward(self, x, d_idx):
        # n*512*512*3
        # Encoder
        for layer in self.encoder:
            x = layer(x)
        # Decoder
        if d_idx == 1:
            for layer in self.decoder_1:
                x = layer(x)
        elif d_idx == 2:
            for layer in self.decoder_2:
                x = layer(x)
        # Classifier
        x = x.view(-1, 3*2*2)
        for layer in self.classifier:
            x = layer(x)
        return x

discriminator = ClassifyCatDog()
if torch.cuda.is_available():
    discriminator = discriminator.cuda()

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer_all = optim.Adam(discriminator.parameters(), lr=0.1, betas=(0.9, 0.999))

def printParam(m):
    for name, param in m.named_parameters():
        print(name, param.data)

for x_i, x in enumerate(dataloader):
    imgs   = x['img']
    labels = x['label']
    imgs   = imgs.cuda()
    labels = labels.cuda()
    
    y  = discriminator(imgs, x_i%2+1)
    loss = criterion(y, labels)
    loss.backward()
    
    print(loss)
    if x_i%2+1 == 1: # path go through decoder_1
        optimizer_all.step()
        optimizer_all.zero_grad()
    else: # path go through decoder_2
        print("decoder_1 step0:")
        printParam(discriminator.decoder_1)
        optimizer_all.step()
#         optimizer_all.zero_grad()
        print("decoder_1 step1:")
        printParam(discriminator.decoder_1)
        optimizer_all.step()
        optimizer_all.zero_grad()
        print("decoder_1 step2:")
        printParam(discriminator.decoder_1)
    
    if x_i == 1:
        break

tensor(0.7197, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6950, device='cuda:0', grad_fn=<NllLossBackward>)
decoder_1 step0:
0.weight tensor([[[[ 0.2185, -0.0138,  0.1966],
          [ 0.1699,  0.0583,  0.1697],
          [ 0.1267,  0.2172, -0.0634]],

         [[ 0.1925,  0.2151, -0.0301],
          [ 0.2544, -0.0651,  0.2653],
          [ 0.0013,  0.0618, -0.0176]],

         [[ 0.1498,  0.0837, -0.0042],
          [ 0.0376,  0.2251, -0.2379],
          [ 0.1500, -0.0341,  0.1122]],

         [[ 0.1791,  0.1259, -0.0328],
          [-0.0073, -0.0265,  0.1859],
          [-0.0341,  0.2337,  0.2162]]],


        [[[ 0.0376, -0.1429, -0.1286],
          [ 0.1939,  0.1836,  0.1059],
          [-0.0140, -0.0231,  0.1421]],

         [[-0.0819, -0.1367, -0.0840],
          [ 0.2254,  0.1810,  0.1778],
          [ 0.2321,  0.1536,  0.0306]],

         [[-0.1590,  0.1090,  0.1535],
          [-0.1991,  0.0952, -0.1795],
          [-0.1225,  0.1179, -0.0391]],

         [[-0.0534,  

In [8]:
print(discriminator.parameters())

<generator object Module.parameters at 0x7f1a26f59cd0>


## Train loop

In [137]:
for epoch in range(10):
    for x_i, x in enumerate(dataloader):
        if x_i%10==9:
            print(x_i)

9
19
29
39
9
19
29
39
9
19
29
39
9
19
29
39
9
19
29
39
9
19
29
39
9
19
29
39
9
19
29
39
9
19
29
39
9
19
29
39
